The code to train the colmap generated images and .bin files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content
!pip install -q plyfile

/content


In [2]:
!wget https://huggingface.co/datasets/twoyoung/christmasTree/resolve/main/3DGS-binary-segmentation.zip
!unzip 3DGS-binary-segmentation.zip

--2024-01-31 22:52:22--  https://huggingface.co/datasets/twoyoung/christmasTree/resolve/main/3DGS-binary-segmentation.zip
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.90, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/b1/3b/b13b0899dd3d7e40b480c3234f0a64b2685475e2d8e25f8b5577189a9973bdd5/0d0ab58ed54c8428feff5d70438cb82ca574c6cb3274acf1fd683d0ac29e400e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%273DGS-binary-segmentation.zip%3B+filename%3D%223DGS-binary-segmentation.zip%22%3B&response-content-type=application%2Fzip&Expires=1707000743&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzAwMDc0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2IxLzNiL2IxM2IwODk5ZGQzZDdlNDBiNDgwYzMyMzRmMGE2NGIyNjg1NDc1ZTJkOGUyNWY4YjU1NzcxODlhOTk3

In [3]:
%cd /content/3DGS-binary-segmentation/
!pip install -q /content/3DGS-binary-segmentation/submodules/diff-gaussian-rasterization
!pip install -q /content/3DGS-binary-segmentation/submodules/simple-knn

/content/3DGS-binary-segmentation
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [4]:
# Download the camera information
!wget https://huggingface.co/datasets/twoyoung/christmasTree/resolve/main/christmasTree.zip
!unzip christmasTree.zip

--2024-01-31 22:57:10--  https://huggingface.co/datasets/twoyoung/christmasTree/resolve/main/christmasTree.zip
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.74, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/b1/3b/b13b0899dd3d7e40b480c3234f0a64b2685475e2d8e25f8b5577189a9973bdd5/27b8ffe66015cbfd3feb95e5e5b1a0da7b039ec309cf8f898fd413e6f882e2fe?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27christmasTree.zip%3B+filename%3D%22christmasTree.zip%22%3B&response-content-type=application%2Fzip&Expires=1707001030&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzAwMTAzMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2IxLzNiL2IxM2IwODk5ZGQzZDdlNDBiNDgwYzMyMzRmMGE2NGIyNjg1NDc1ZTJkOGUyNWY4YjU1NzcxODlhOTk3M2JkZDUvMjdiOGZmZTY2MDE1Y2JmZDNmZWI

In [5]:
# Download the masks
!wget https://huggingface.co/datasets/twoyoung/christmasTree/resolve/main/masks.zip
!unzip masks.zip

--2024-01-31 22:57:48--  https://huggingface.co/datasets/twoyoung/christmasTree/resolve/main/masks.zip
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.114, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/b1/3b/b13b0899dd3d7e40b480c3234f0a64b2685475e2d8e25f8b5577189a9973bdd5/0d04a1ed90593bcf6af470733baefbc3f7652084ac4178feb035783094fb0fb5?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27masks.zip%3B+filename%3D%22masks.zip%22%3B&response-content-type=application%2Fzip&Expires=1707001068&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzAwMTA2OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2IxLzNiL2IxM2IwODk5ZGQzZDdlNDBiNDgwYzMyMzRmMGE2NGIyNjg1NDc1ZTJkOGUyNWY4YjU1NzcxODlhOTk3M2JkZDUvMGQwNGExZWQ5MDU5M2JjZjZhZjQ3MDczM2JhZWZiYzNmNzY1Mj

In [6]:
# Download the pre-trained 3d gaussian splatting model with p and rgb attribute added
!wget https://huggingface.co/datasets/twoyoung/christmasTree/resolve/main/pre-trained_3d_gaussian_splatting_model_with_p_and_rgb_added.zip
!unzip pre-trained_3d_gaussian_splatting_model_with_p_and_rgb_added.zip

--2024-01-31 22:57:52--  https://huggingface.co/datasets/twoyoung/christmasTree/resolve/main/pre-trained_3d_gaussian_splatting_model_with_p_and_rgb_added.zip
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.114, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/b1/3b/b13b0899dd3d7e40b480c3234f0a64b2685475e2d8e25f8b5577189a9973bdd5/fa60fd3df37f296af8aecf16109bf0ebb616d8a3f04590e5b7d0a4ef08e4a166?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pre-trained_3d_gaussian_splatting_model_with_p_and_rgb_added.zip%3B+filename%3D%22pre-trained_3d_gaussian_splatting_model_with_p_and_rgb_added.zip%22%3B&response-content-type=application%2Fzip&Expires=1707001072&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzAwMTA3Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpb

In [7]:
!python train.py -s /content/3DGS-binary-segmentation/christmasTree --kmask_path /content/3DGS-binary-segmentation/masks --model_path /content/3DGS-binary-segmentation/pre-trained_3d_gaussian_splatting_model_with_p_and_rgb_added/c95e51cf-3/point_cloud/iteration_10000 --eval

2024-01-31 22:58:03.349485: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 22:58:03.349536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 22:58:03.351250: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 22:58:03.358658: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 22:58:04.390247: W tensorflow/compiler/tf2

In [ ]:
# prompt: move output folder to drive/MyDrive/

!mv /content/gaussian-splatting/output /content/drive/MyDrive/
